<a href="https://colab.research.google.com/github/NiclasFenton-Wiegleb/schlager-lyrics-bot/blob/main/Schlager_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/NiclasFenton-Wiegleb/schlager-lyrics-bot.git

Cloning into 'schlager-lyrics-bot'...
remote: Enumerating objects: 1890, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 1890 (delta 20), reused 32 (delta 8), pack-reused 1838
Receiving objects: 100% (1890/1890), 98.35 MiB | 19.56 MiB/s, done.
Resolving deltas: 100% (112/112), done.
Updating files: 100% (3282/3282), done.


In [3]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [1]:
!conda --version

conda 23.1.0


In [2]:
!conda create -n schlager_bot --file ./schlager-lyrics-bot/req.txt

Solving environment: / failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: - \ failed

PackagesNotFoundError: The following packages are not available from current channels:

  - mkl_random==1.2.2=py38h51133e4_0
  - tqdm==4.64.1=py38h06a4308_0
  - ruamel.yaml.clib==0.2.6=py38h5eee18b_1
  - jupyter_core==5.1.1=py38h06a4308_0
  - mkl==2021.4.0=h06a4308_640
  - jupyter_console==6.4.4=py38h06a4308_0
  - jupyter_server==1.23.4=py38h06a4308_0
  - numpy==1.23.5=py38h14f4228_0
  - lcms2==2.12=h3be6417_0
  - munkres==1.1.4=py_0
  - prometheus_client==0.14.1=py38h06a4308_0
  - glib==2.69.1=he621ea3_2
  - prompt-toolkit==3.0.36=py38h06a4308_0
  - tomli==2.0.1=py38h06a4308_0
  - libbrotlienc==1.0.9=h5eee18b_7
  - seaborn==0.12.2=py38h06a4308_0
  - toolz==0.12.0=py38h06a4308_0
  - notebook-shim==0.2.2=py38h06a4308_0
  - cffi==1.15.1=py38h5eee18b_3
  - fontconfig==2.14.1=h52c9d5c_1
  - markupsafe==2.1.1=py38h7f8727e_0
  - mpmath==1.2.1=py

In [5]:
!conda env update -n base -f  ./schlager-lyrics-bot/req.txt


Solving environment: \ | failed

ResolvePackageNotFound: 
  - importlib-metadata==4.11.3=py38h06a4308_0
  - jpeg==9e=h7f8727e_0
  - tomli==2.0.1=py38h06a4308_0
  - toolz==0.12.0=py38h06a4308_0
  - filelock==3.9.0=py38h06a4308_0
  - pkgutil-resolve-name==1.3.10=py38h06a4308_0
  - zstandard==0.18.0=py38h5eee18b_0
  - prometheus_client==0.14.1=py38h06a4308_0
  - markupsafe==2.1.1=py38h7f8727e_0
  - torchaudio==2.0.0=py38_cpu
  - toml==0.10.2=pyhd3eb1b0_0
  - libxslt==1.1.35=h4e12654_0
  - libdeflate==1.8=h7f8727e_5
  - nbformat==5.7.0=py38h06a4308_0
  - notebook-shim==0.2.2=py38h06a4308_0
  - pillow==9.3.0=py38h6a678d5_2
  - python-dateutil==2.8.2=pyhd3eb1b0_0
  - pyasn1==0.4.8=pypi_0
  - websocket-client==0.58.0=py38h06a4308_4
  - jupyterlab_server==2.16.5=py38h06a4308_0
  - contourpy==1.0.5=py38hdb19cb5_0
  - mkl-service==2.4.0=py38h7f8727e_0
  - lerc==3.0=h295c915_0
  - webencodings==0.5.1=py38_1
  - brotlipy==0.7.0=py38h27cfd23_1003
  - tqdm==4.64.1=py38h06a4308_0
  - colorama==0.

In [6]:
!git push origin

fatal: not a git repository (or any of the parent directories): .git


# Training the Schlager Bot Language Model

To train the model that will ultimately generate Schlager lyrics, we first train a BERT model from scratch on our text dataset composed of Schlager lyrics and some generic German text from an open source NLP training dataset. The latter serves to provide grammatical structure and vocabulary that the lyrics alone lack.

Pre-training on transformers can be done with self-supervised tasks. In this case we will use Masked Language Modeling (MLM), where a certain percentage of the tokens in a sentence is masked and the model is trained to predict those masked words. One of the advantages of this method is that it can see the position information of the whole sentence - both for the masked and visible part.

First we need to import the relevant dependencies.

In [ ]:
import transformers
import json
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
import datasets
import os
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizerFast
from itertools import chain

Before we can start putting our model together, we need to preprocess the training data and split it into training and test sub-sets. Separating these out first avoids the model seeing the test data during training and risking overfitting.

In [ ]:
#Loading the lyrics data as the dataset

#Iterating through the files in the directory and adding the names to the files variable

files = []

directory = "./lyrics"

for filename in os.scandir(directory):
    if filename.is_file():
        files.append(filename.path)

dataset = datasets.load_dataset("text", data_files= files, split= "train")

In [ ]:
#Now we split the dataset into training (90%) and testing (10%)

d = dataset.train_test_split(test_size= 0.1)
d["train"], d["test"]

Now that the dataset is loaded and split into training and test data, it is time to train the tokenizer. To achieve this, we need to write our dataset into text files - keeping training and test data separate.

In [ ]:
def dataset_to_text(dataset, output_filename="data.txt"):
  """Utility function to save dataset text to disk,
  useful for using the texts to train the tokenizer
  (as the tokenizer accepts files)"""
  with open(output_filename, "w") as f:
    for t in dataset["text"]:
      print(t, file=f)

# save the training set to train.txt
dataset_to_text(d["train"], "./model/model_data/train.txt")
# save the testing set to test.txt
dataset_to_text(d["test"], "./model/model_data/test.txt")

Next we define some parameters of the tokenizer. The training file indicates the data we're passing to the tokenizer for training. This could be a list of files too. vocab_size is the vocabulary size of tokens, while max_length is the maximum sequence length.

truncate_longer_samples is a boolean indicating whether we truncate sentences longer than the length of max_length, if it's set to False, we won't truncate the sentences, we group them together and split them by max_length, so all the resulting sentences will have the length of max_length.

In [ ]:
special_tokens = [
  "[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"
]

training_file = ["./model/model_data/train.txt"]
# 30,522 vocab is BERT's default vocab size
vocab_size = 30_522
# maximum sequence length, lowering will result to faster training (when increasing batch size)
max_length = 512
# whether to truncate
truncate_longer_samples = False

We are now ready to train the tokenizer. BERT's default tokenizer is WordPiece and, therefore, we initialize the BertWordPieceTokenizer() tokenizer class from the tokenizers library and use the train() method to train it. It will take several minutes to finish. We then need to save the tokenizer to a directory.

In [ ]:
#Initialize the WordPiece tokenizer
tokenizer = BertWordPieceTokenizer()

#Train the tokenizer
tokenizer.train(files=training_file, vocab_size=vocab_size, special_tokens=special_tokens)

#Enable truncation up to the maximum 512 tokens
# tokenizer.enable_truncation(max_length=max_length)

model_path = "./model/tokenizer/"

#Save the tokenizer to directory
tokenizer.save_model(model_path)

#Dumping some of the tokenizer config to config file,
#including special tokens, whether to lower case and the maximum sequence length
with open(os.path.join(model_path, "config.json"), "w") as f:
  tokenizer_cfg = {
      "do_lower_case": True,
      "unk_token": "[UNK]",
      "sep_token": "[SEP]",
      "pad_token": "[PAD]",
      "cls_token": "[CLS]",
      "mask_token": "[MASK]",
      "model_max_length": max_length,
      "max_len": max_length,
  }
  json.dump(tokenizer_cfg, f)

The tokenizer.save_model() method saves the vocabulary file into that path, we also manually save some tokenizer configurations, such as special tokens:

- unk_token: A special token that represents an out-of-vocabulary token, even though the tokenizer is a WordPiece tokenizer, the unk tokens are not
- impossible, but rare.
- sep_token: A special token that separates two different sentences in the same input.
- pad_token: A special token that is used to fill sentences that do not reach the maximum sequence length (since the arrays of tokens must be the same size).
- cls_token: A special token representing the class of the input.
- mask_token: This is the mask token we use for the Masked Language Modeling (MLM) pretraining task.

In [ ]:
#Let's load the tokenizer
model_path = "./model/tokenizer/"
tokenizer = transformers.BertTokenizerFast.from_pretrained(model_path)

With the tokenizer ready to be taken into operation, we can now tokenize our dataset.

In [ ]:
#Load train and test data
train_data = datasets.load_dataset("text", data_files= "./model/model_data/train.txt", split= "train")
test_data = datasets.load_dataset("text", data_files= "./model/model_data/test.txt", split= "train")
#Tokenizing the training dataset
train_dataset = train_data.map((lambda x: tokenizer(x["text"], return_special_tokens_mask=True)), batched= True)

#Tokenizing the test dataset
test_dataset = test_data.map((lambda x: tokenizer(x["text"], return_special_tokens_mask=True)), batched= True)

In [ ]:
# Remove other columns, and remain them as Python lists

test_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])
train_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])

In [ ]:
# Main data processing function that will concatenate all texts from our dataset and generate chunks of
# max_seq_length.
# grabbed from: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py

max_length = 256

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= max_length:
        total_length = (total_length // max_length) * max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + max_length] for i in range(0, total_length, max_length)]
        for k, t in concatenated_examples.items()
    }
    return result

train_dataset = train_dataset.map(group_texts, batched=True,
                                desc=f"Grouping texts in chunks of {max_length}")
test_dataset = test_dataset.map(group_texts, batched=True,
                                desc=f"Grouping texts in chunks of {max_length}")

In [ ]:
# convert them from lists to torch tensors
train_dataset.set_format(type='torch')